In [1]:

import numpy as np 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16


In [2]:
b_size = 5

train_datagen = ImageDataGenerator(rescale=1./255, 
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory('train', target_size = (150,150), 
                                                 batch_size = b_size,
                                                 class_mode='binary')

Found 160 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('test', target_size = (150,150), 
                                                 batch_size = b_size,
                                                 class_mode='binary')

Found 120 images belonging to 2 classes.


In [4]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(150, 150, 3))
transfer_model.trainable = False
#일반적으로 model의 top부분은 문제해결을 위한 분류기로 되어있다. 
#우리가 그동안 코드를 작성하면서 하향식으로 내려갔기 떄문에 순서가 다르지만 아무튼 그러하다....
transfer_model.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [5]:
finetune_model = Sequential()
#finetune : 미세조정
finetune_model.add(transfer_model)
#하단에 transfer model을 넣겠다는 것임
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(2, activation='softmax'))
finetune_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                524352    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 15,239,170
Trainable params: 524,482
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
finetune_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer = optimizers.Adam(learning_rate=0.0002),
             metrics = ['accuracy'])

steps_train = len(train_generator)
steps_test = len(test_generator)


hist = finetune_model.fit(train_generator, steps_per_epoch=steps_train, epochs = 20, 
                 validation_data=test_generator, validation_steps=steps_test)

Epoch 1/20
32/32 [==============================] - 14s 448ms/step - loss: 0.5454 - accuracy: 0.7250 - val_loss: 0.4403 - val_accuracy: 0.7917
Epoch 2/20
32/32 [==============================] - 14s 432ms/step - loss: 0.3192 - accuracy: 0.9000 - val_loss: 0.2829 - val_accuracy: 0.9000
Epoch 3/20
32/32 [==============================] - 14s 434ms/step - loss: 0.2304 - accuracy: 0.9312 - val_loss: 0.2197 - val_accuracy: 0.9167
Epoch 4/20
32/32 [==============================] - 14s 431ms/step - loss: 0.1929 - accuracy: 0.9312 - val_loss: 0.4134 - val_accuracy: 0.8083
Epoch 5/20
32/32 [==============================] - 13s 420ms/step - loss: 0.1432 - accuracy: 0.9563 - val_loss: 0.1635 - val_accuracy: 0.9583
Epoch 6/20
32/32 [==============================] - 13s 420ms/step - loss: 0.1717 - accuracy: 0.9375 - val_loss: 0.1617 - val_accuracy: 0.9417
Epoch 7/20
32/32 [==============================] - 14s 428ms/step - loss: 0.1349 - accuracy: 0.9625 - val_loss: 0.2819 - val_accuracy: 0.8917